# GREED MODE 

**Github repo link**: https://github.com/ff-falco/AIFPROJ2023

For the project we choose to build a brand new game mode that can be played in NetHack, the Greed Mode.  

In this particular endeavor, the agent is tasked with navigating through five levels while maximizing both wealth and health.
At each floor the agent goes through a shopping phase in the shop, where he can buy items for the battle phase in the battlefield.  
In the battlefield there are enemies, whose strenght depends on the current level, and gold, the currency of the shop, that can be used in the next floor to buy  even better items.


To assess the agent's performance, we employ three key metrics:
  
* **Healtiness**: the remaining number of hit points. To restore them the shop provides health potions and armors to preserve them.
  
* **Richness**: the remaining amount of money at the agent's disposal. A proficient agent can strategically manage its resources, avoiding the necessity to purchase the best items at every opportunity.

  
* **Levels completed**: the last metric that is important to us are the levels completed by the agent. Indeed, we have to keep in mind, while we want to preserve our health and have a large quantity of money, those are mainly than a means to reach higher levels.
  
  
To play this game mode, we had to create a new environment from scratch because the standard NetHack shop customization options were limited. Our process involves randomly generating the shop and battlefield components for each level. These parts are then combined into a single des file, making it a usable level for simulations using the Gym libraries. This straightforward approach enables a customized and dynamic gaming experience in the Greed Mode.

**NOTE** For the execution of this notebook it is expected to use the environment used for the handson 2, since we use for the agent a prolog knowledge base.
 

# LEVEL GENERATION

In [ ]:
import gym
import random
import math
import matplotlib.pyplot as plt

from minihack import LevelGenerator
import run

import gym
from run import *
from algorithms import  a_star
from utils import process_state, perform_action,extract_monsters, set_health, check_death, display_inventory,parse_predicate,show_match,make_confuse
from utilsbattle import actions_from_path,get_money_location, get_stair_location, get_target_location, get_reference_des,NUM_TO_DIR_MAP,DIR_TO_NUM_MAP,get_closest_gold,chebyshev_distance
#import utilsbattle
import random
import math
import matplotlib.pyplot as plt
from minihack import LevelGenerator
from pyswip import Prolog




In [ ]:
def render_des_file(des_file):

    env = gym.make('MiniHack-Skill-Custom-v0',
               character="sam-hum-neu-mal",
               observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel'),
               des_file=des_file)

    obs = env.reset()
    env.render()
    plt.imshow(obs['pixel'][0:1000, 0:1000])

## SHOP GENERATION

Now we define the function that creates the store des files, the map is default but the items are randomly generated with this logic:  
  
* A Healing Item (healing potion) is always present.  


* Two to five items are present on the right, these are always different (sword and bow, bow and armor, sword and armor, sword and best sword)  
* The pricing of the items it's fixed based on their type. An **healing potion** costs **5**, a **weapon** costs **15** and an **armor** costs **20**. It's important to note that while these costs are inherent to the game, they are not explicitly modeled within the environment. Instead, they are documented in the knowledge base, especially considering that we are not utilizing the inherent NetHack shop mechanics
* The weapons and armors change the agent stats, making more enemies beatable. The healing potion instead heals the hp of the agent (and, very rarely, increases its maximum HP)

**THE AVAILABLE ITEMS ARE THE FOLLOWING**


-**Available items** are:
**healing** (life potion)
**'weapon'** (close range)
**'arrow'** (long range)
**'armor'** (increasing resistance)

-**Available swords** are :
'spear',
'dwarvish spear',
'morning star',
'elven broadsword',
'trident'.

-**Available arrows** are:
'orcish arrow',
'silver arrow',
'arrow',
'elven arrow',
'ya'.

-**Available armors** are :
'red dragon scales',
'scale mail',
'elven mithril-coat',
'dwarvish mithril-coat',
'red dragon scale mail'.






The reason for choosing the following items is based on an investigation done by us on the nethack wiki:

* The weapons are chosen among the ones that are considered good for the caveman (the starting role of our agent).


* Both the weapons and armors are chosen with a increasing damage or AC, chosen by respectively looking at https://nethackwiki.com/wiki/Weapon and https://nethackwiki.com/wiki/Armor


All items are ordered from weakest to strongest.

In [ ]:
AVAIABLE_OBJECTS=run.AVAIABLE_OBJECTS
AVAIABLE_ARROWS=run.AVAIABLE_ARROWS
AVAIABLE_WEAPONS=run.AVAIABLE_WEAPONS
AVAIABLE_ARMORS=run.AVAIABLE_ARMORS
SYMBOLS=run.SYMBOLS
avaiableitems=run.avaiableitems


In [ ]:
print(run.AVAIABLE_OBJECTS)
print(run.AVAIABLE_WEAPONS)
print(run.AVAIABLE_ARMORS)



We define the function that adds a random object (dependent on the seed entered) to the given location.  


* In higher levels better weapons are spawned.  


* The items type offered are equal but with more quality if all items are already spawned (only in the case five items are present)  
* All the items in the shop can be purchased by picking them up and updating the Knowledge base ()



In [ ]:
def addobject(lvl: LevelGenerator, seed:int, posx:int ,posy:int,others: [str]) ->str:
    """
    lvl is the Level Generator object, where we have to add items.
    seed is the starting number for the level generation, it has to be included in a range from 0 to 200.
    posx and posy are the coordinates for the item to spawn in the shop, it depends from the avaiable quantity.
    others is the list of the name of the others item already spawned.
    addobject returns the name of the spawned item, useful to update the others list mentioned before. 
    """
    #For the shop we use seeds that are multiple of the original seed
    levelnumber =math.floor(seed/200)
    quality=levelnumber-1
    remainingitems=avaiableitems.copy()
    #If the shop has 5 items there is an item from a highest quality, or lowest inthe level 5 case, we refresh the array with the remaining avaible items.
    for i in others:
        if i in remainingitems :
            remainingitems.pop(remainingitems.index(i))

    if remainingitems.__len__()==0:
        remainingitems=avaiableitems.copy()
        if levelnumber==5:
            quality=quality-1
        else: quality=quality+1

    choice=seed%remainingitems.__len__()
    choosen=remainingitems[choice]
    pos=(posx,posy)

    if choosen=='HEALING':
        choosenitem=AVAIABLE_OBJECTS[0]
        choosensymbol=SYMBOLS[0]
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=pos)

    elif choosen=='ARROWS':
        choosenitem=AVAIABLE_ARROWS[quality]
        choosensymbol=SYMBOLS[1]
        #In the arrow case we are spawning three of them, it's enough to kill every mob
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=(posx,posy))
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=(posx,posy+1))
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=(posx,posy+2))

    elif choosen=='WEAPON':
        choosenitem=AVAIABLE_WEAPONS[quality]
        choosensymbol=SYMBOLS[2]
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=pos)
        """
        lvl.add_line("OBJECT:('"+choosensymbol+"', \""+choosenitem+"\"),("+str(pos[0])+","+str(pos[1])+")")
        #, blessed ,+"+str(powerup))"""

    else:
        choosenitem=AVAIABLE_ARMORS[quality]
        choosensymbol=SYMBOLS[3]
        lvl.add_object(name=choosenitem,symbol=choosensymbol,place=pos)
        
    return choosen
    

Now we define the funcion that fills the shop with a number of items depending on the level:  

* Level 1: 3 (80%), 4 (15%) 5 (5%)  

* Level 2: 3 (50%), 4 (40%) 5 (10%)  
* Level 3: 3 (30%), 4 (50%) 5 (20%)  
* Level 4: 3 (20%), 4 (30%) 5 (50%)  
* Level 5: 3 (5%), 4 (15%) 5 (80%)  

We choose to increase probability of more items in the later levels because the enemies become harder and better items can be useful to reach the end.


In [ ]:

def fillwithobj(lvl: LevelGenerator,seed: int,levelnumber:int):
    """
    fillwithobj manage the number of the items spawned depending of the number, further details are explained in the levelgenerator notebook.
    """
    random.seed(seed)
    numitem=random.randint(3,5)  



    if levelnumber==1:
        if seed<=160:
            numitem=3
        elif seed<=190:
            numitem=4
        else: 
            numitem=5

    elif levelnumber==2:
        if seed<=100:
            numitem=3
        elif seed<=180:
            numitem=4
        else: 
            numitem=5

    if levelnumber==3:
        if seed<=60:
            numitem=3
        elif seed<=160:
            numitem=4
        else: 
            numitem=5

    if levelnumber==4:
        if seed<=40:
            numitem=3
        elif seed<=100:
            numitem=4
        else: 
            numitem=5

    if levelnumber==5:
        if seed<=10:
            numitem=3
        elif seed<=40:
            numitem=4
        else: 
            numitem=5
            
    others=[]
    #In the first level a healing potion is guaranteed, the other choices are pseudo-random (depending on the seed)  
    if numitem==3:
        if levelnumber==1:
            others=['HEALING']
            lvl.add_object(name='healing',symbol='!',place=(4,3))
        else:
            others.append(addobject(lvl,seed,4,3,others))
        others.append(addobject(lvl,seed,6,3,others))
        others.append(addobject(lvl,seed,8,3,others))

    elif numitem==4:
        if levelnumber==1:
            others=['HEALING']
            lvl.add_object(name='healing',symbol='!',place=(3,3))
        else:
            others.append(addobject(lvl,seed,3,3,others))
        others.append(addobject(lvl,seed,5,3,others))
        others.append(addobject(lvl,seed,7,3,others))
        others.append(addobject(lvl,seed,9,3,others))

    elif numitem==5:
        if levelnumber==1:
            others=['HEALING']
            lvl.add_object(name='healing',symbol='!',place=(2,3))
        else:
            others.append(addobject(lvl,seed,2,3,others))
        others.append(addobject(lvl,seed,4,3,others))
        others.append(addobject(lvl,seed,6,3,others))
        others.append(addobject(lvl,seed,8,3,others))
        others.append(addobject(lvl,seed,10,3,others))








Finally we define the function that creates the shop .des file and displays it on the screen.  


The levelnumber is used to generate levels depending on the floor the agent has reached. Higher floors are designed to correspond with better items available in the shop, with a greater variety of choices.

In the latest version, the shopkeeper, always present in the standard NetHack shop, has been eliminated. This decision stems from the fact that the shopkeeper's role is rendered redundant in our context. This is due to our departure from utilizing the payment mechanism inherent in NetHack, opting instead to simulate it within our knowledge base.

Once the purchases are done the agent can pick up the Skeleton Keys placed in front of the exit (we put two of the because they can break, although very rarely) and open the way to the battlefield area.  


The seed must be choosen from 1 to 200 and the levelnumber must be from 1 to 5.

In [ ]:

def createshop(levelnumber: int,startingseed: int):
    map="""||||||||||||||||
|...............
|...........||||
|...........|    
|...........|    
|...........|   
|...........|     
|||||||||||||"""
    seed=startingseed
    lvl=LevelGenerator(map=map,lit=True,flags=['premapped','shortsighted'])
    lvl.add_line("#LEVEL: "+str(levelnumber))
    lvl.add_line("#SEED: "+str(seed))
    lvl.set_start_pos((1,6))
    #lvl.add_monster(name='shopkeeper',place=(6,1))
    lvl.add_object(name='skeleton key',symbol='(',place=(11,1)) #Skeleton Key
    lvl.add_object(name='skeleton key',symbol='(',place=(11,2))
    lvl.add_door("locked", place=(12,1))
    fillwithobj(lvl,seed,levelnumber)
    return lvl.get_des()

seed=random.randint(1,200)
level=random.randint(1,5)
render_des_file(createshop(level,seed))

## BATTLEFIELD GENERATION

In this section we show how we generate the battlefield, the area where our agent will find gold and monstersto fight.


For the battlefield we chose 5 possible maps, with a random generation of monsters and coins.
In each possible level it can happen:  
* **Boss room**: a room containing only one enemy, the strongest for that level, that defends lots of coins;


* **Maze room**: a maze. In this room the agent has to perform pathfinding to find the exit and the gold;
* **Room**: an open room with enemies and coins;
* **Miniboss room**: the room shown during the presentation, where there's a miniboss defending lots of gold, and enemies defending the exit;
* **BONUS ROOM**: a very rare room, (0.5% chance of being generated) where there are only coins.  


Before showcasing the rooms, we establish a comprehensive list of monsters, arranged in ascending order based on their difficulty. The criteria guiding our selection process are straightforward:

* Monsters are chosen for each level in accordance with their increasing difficulty, as outlined in the NetHack wiki for each respective monster.
  
* Selected monsters must not possess the ability to break or open doors.

* Monsters are restricted from engaging in item-related activities such as stealing, picking up, or interacting with items in any manner. This ensures a controlled and predictable environment for the agent to navigate through without unexpected complications involving items.


In [ ]:
MOB = run.MOB
print(run.MOB)

MINIBOSS=run.MINIBOSS
print(run.MINIBOSS)

BOSS=run.BOSS
print(run.BOSS)


In [ ]:
minibossmap="""                    ||||||||||||
                    |..........|
                    |.||||||...|
                    |.|........|
                    |.|||||....|
                    |.....|....|
                    |||||||....|
                    |..........|
                    |.|||||....|
                    |.|...|....|
                    |.|...|....|
                    ..|...|....|
                    |||.||||...|
                    |......|...|
                    |......|...|
                    |..........|
                    |......|...|
                    ||||||||||||"""




bossmap="""                          |||||      
                          |...|      
                          |...|      
                    ||||||||.||||    
                    |...|.......|    
                    |...|.......|||||
                    |...|.......|...|
                    ................|
                    |...|.......|...|
                    |...|.......|||||
                    |...|.......|    
                    |||||||||||||"""    


roommap="""                    |||||||||||||||
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    |.............|
                    ..............|
                    |||||||||||||||"""

mazebasemap="""                    |||||||||||||||||||||||||
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    ..                     .|
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    |.                     .|
                    |||||||||||||||||||||||||"""

bonusroommap="""                    |||||||
                    |.....|
                    ......|
                    |.....|
                    |||||||
                    
                    
                    
                    
                    
                    """



**Bonus room**: small room with 15 gold that can be taken without any difficulty.

In [ ]:
def bonusroom()->str:
    lvl=LevelGenerator(lit=True,flags=['premapped'],map=bonusroommap)
    lvl.set_start_pos((21,2))
    lvl.add_stair_down(place=(24,1))
    lvl.add_line("GOLD: 5,(22,2)")
    lvl.add_line("GOLD: 5,(23,2)")
    lvl.add_line("GOLD: 5,(24,2)")
    
    return lvl.get_des()

render_des_file(bonusroom())


**Boss room**: a room with a boss and 20 gold of reward for its death.

In [ ]:
def bossroomdes(levelnumber: int)->str:
    lvl=LevelGenerator(map=bossmap,lit=True,flags=['premapped'])
    lvl.set_start_pos((21,7))
    lvl.add_stair_down(place=(28, 1))
    lvl.add_line("GOLD: 5,(34,7)")
    lvl.add_line("GOLD: 5,(35,6)")
    lvl.add_line("GOLD: 5,(35,7)")
    lvl.add_line("GOLD: 5,(35,8)")
    lvl.add_monster(name=BOSS[levelnumber-2],place=(29,7))
    
    return lvl.get_des()

render_des_file(bossroomdes(2))


**Miniboss room**: the room shown in the presentation, three Mobs defend the exit, one defends 5 gold and a Miniboss defends 15 gold.

In [ ]:
def minibossroomdes(levelnumber: int)->str:
    lvl=LevelGenerator(map=minibossmap,lit=True,flags=['premapped'])
    lvl.set_start_pos((21,11))
    lvl.add_stair_down(place=(30, 1))
    
    #MOB ZONE
    lvl.add_monster(name=MOB[levelnumber-1],place=(28,1))
    lvl.add_monster(name=MOB[levelnumber-1],place=(29,2))
    lvl.add_monster(name=MOB[levelnumber-1],place=(30,3))

    #MOB ZONE (optional)
    lvl.add_monster(name=MOB[levelnumber-1],place=(21,1))
    #MOB REWARD
    lvl.add_line("GOLD: 5,(25,5)")

    #MINIBOSS ZONE (optional)
    lvl.add_monster(name=MINIBOSS[levelnumber-2],place=(23,13))
    #MINIBOSS REWARD
    lvl.add_line("GOLD: 5,(23,10)")
    lvl.add_line("GOLD: 5,(24,10)")
    lvl.add_line("GOLD: 5,(25,10)")

    return lvl.get_des()

render_des_file(minibossroomdes(2))


**Common room**: room without obstacles; mobs and gold are randomly distributed.

In [ ]:
def roomroomdes(seed: int,levelnumber : int)->str:
    lvl=LevelGenerator(map=roommap,lit=True,flags=['premapped'])
    lvl.set_start_pos((21,12))
    lvl.add_stair_down(place=(33, 1))
    random.seed(seed)

    #MUTUALLY EXCLUSIVE RANDOMNESS----
    pseudoarray=[]
    for i in range(1,13):
        pseudoarray.append(i)
    pseudomatrix=[]
    for i in range(0,10):
        pseudomatrix.append(pseudoarray)
    temporaryseed=seed
    def pseudorandompos (seed:(int))->(int,int):
        random.seed(seed)
        a=random.randint(0,9)
        b=random.choice(pseudomatrix[a])
        pseudomatrix[a].pop(pseudomatrix[a].index(b))
        return (a+22,b)
    #---------------------------------    

    for i in range(0,4):
        lvl.add_line("GOLD: 5,"+str(pseudorandompos(temporaryseed)))
        temporaryseed=temporaryseed+1

    for i in range(0,6):
        lvl.add_monster(name=MOB[levelnumber-1],place=pseudorandompos(temporaryseed))
        temporaryseed=temporaryseed+1
  
    return lvl.get_des()

render_des_file(roomroomdes(random.randint(1,200),1))





**Maze Room**: a small Labirinth with three Mobs in moving in it. At the end of the level there are 15 gold as reward.

In [ ]:
def mazeroomdes(levelnumber:int )->str:
    lvl=LevelGenerator(map=mazebasemap,lit=True,flags=['premapped'])
    lvl.set_start_pos((21,4))

    lvl.add_stair_down(place=(43, 4))
    
    lvl.add_mazewalk(coord=(21,4), dir='east')

    lvl.add_monster(MOB[levelnumber-1],place=(43,3))
    lvl.add_monster(MOB[levelnumber-1],place=(43,2))
    lvl.add_monster(MOB[levelnumber-1],place=(43,8))
    lvl.add_monster(MOB[levelnumber-1],place=(43,9))

    lvl.add_line("GOLD: 5,(43,5)")
    lvl.add_line("GOLD: 5,(43,6)")
    lvl.add_line("GOLD: 5,(43,7)")
 
    return lvl.get_des()


render_des_file(mazeroomdes(2))
render_des_file(mazeroomdes(3))
render_des_file(mazeroomdes(4))


The probabilities of a room spawning are the following:


* Common Room = 50%
* Miniboss Room = 30% 
* Boss Room = 10% 
* Maze Room = 9,5% 
* Bonus Room= 0,5% 

**However**:
* The **first level** is guaranteed to be a Common Room;
* The **last level** is guaranteed to be a Boss Room.



In [ ]:
def createbattlefield(levelnumber:int ,startingseed:int )->(str,str):
    if levelnumber==1:
        return roomroomdes(startingseed, levelnumber),"roomroom"
    elif levelnumber==5:
        return bossroomdes(5),"bossroom"
    else:
        random.seed(startingseed*levelnumber)
        choice=random.randint(0,200)
        if(choice<=100):
            return (roomroomdes(startingseed,levelnumber),"roomroom")
        elif(choice<=160):
            return (minibossroomdes(levelnumber),"minibossroom")
        elif(choice<=180):
            return (bossroomdes(levelnumber),"bossroom")
        elif(choice<=199):
            return (mazeroomdes(levelnumber),"mazeroom")
        else:
            return (bonusroom(),"bonusroom")

    
render_des_file(createbattlefield(levelnumber=random.randint(1,5),startingseed=random.randint(1,200))[0])

## Merging the shop and the battlefield

In the last part of the level generation notebook we are going to merge the generated shop and battlefield to obtain the .des file of the completed level.  
The required arguments are the level number and a seed.

In [ ]:

print(run.mapmerger(run.takemap(createshop(1,1)),run.takemap(bonusroom())))
print('\n\n\n')
print(run.mapmerger(run.takemap(createshop(2,2)),run.takemap(bossroomdes(2))))
print('\n\n\n')
print(run.mapmerger(run.takemap(createshop(3,2)),run.takemap(minibossroomdes(3))))
print('\n\n\n')
print(run.mapmerger(run.takemap(createshop(4,2)),run.takemap(roomroomdes(2,4))))
print('\n\n\n')
print(run.mapmerger(run.takemap(createshop(4,2)),run.takemap(roomroomdes(2,4))))

Once the two parts of the levelare generated, we identify the merging points: the exit of the shop and the spawn point of the battlefield. These points are then connected through a corridor, whose ends are secured by two locked doors, both requiring keys for access

Originally, we considered implementing a portal in the shop to facilitate the agent's teleportation to the battlefield. However, upon further investigation, we encountered challenges in controlling the teleportation process through the portal. Subsequently, we opted for a more straightforward solution involving two locked doors.

In [ ]:



#completemapdes returns the des file of the completed level with shop and battlefield merged in a unique level, they are connected by a simple corridor and separated by two locked door that can be unlocked by the skeleton keys in the shop.
def completemapdes(levelnumber,startingseed,custom=None):
    if custom==None:
        battlefield,custom=createbattlefield(levelnumber,startingseed)
    elif custom=="boss":
        battlefield=bossroomdes(levelnumber)
    elif custom=="miniboss":
        battlefield=minibossroomdes(levelnumber)
    elif custom=="room":
        battlefield=roomroomdes(startingseed,levelnumber)
    elif custom=="maze":
        battlefield=mazeroomdes(levelnumber)
    elif custom=="bonus":
        battlefield=bonusroom()
    else:
        print("choosen a wrong custom map")
        return None

    portal=run.incomingextractor(battlefield)
    battlefield=portal[0]
    incominglocation=portal[1]
    
    battlefieldheader=run.headerextractor(battlefield)
    shop=createshop(levelnumber,startingseed)
    shopheader=run.headerextractor(shop)
    lvl=LevelGenerator(map=run.mapmerger(run.takemap(shop),run.takemap(battlefield)),lit=True,flags=['premapped','shortsighted'])
    lvl.add_line(shopheader)
    lvl.add_object(name="amulet of ESP",symbol="\"",place=(1,1))
    lvl.add_line(battlefieldheader)
    lvl.add_door("locked", place=incominglocation)
    lvl.fill_terrain("rect",".",15,1,15,incominglocation[1])
    lvl.fill_terrain("rect",".",15,incominglocation[1],incominglocation[0]-1,incominglocation[1])
    return lvl.get_des(),custom




render_des_file(completemapdes(levelnumber=random.randint(2,5),startingseed=random.randint(1,200),custom="boss")[0])
render_des_file(completemapdes(levelnumber=random.randint(2,4),startingseed=random.randint(1,200),custom="miniboss")[0])
render_des_file(completemapdes(levelnumber=random.randint(2,4),startingseed=random.randint(1,200),custom="bonus")[0])
render_des_file(completemapdes(levelnumber=random.randint(1,4),startingseed=random.randint(1,200),custom="room")[0])
render_des_file(completemapdes(levelnumber=random.randint(2,4),startingseed=random.randint(1,200),custom="maze")[0])



IN ORDER TO CREATE A NEW VALID DES CALL: completemapdes(1,20)
IF A SPECIFIC TYPE OF LEVEL IS REQUIRED ADD THE ARGUMENT ("room","maze","boss","miniboss","bonus") AS THE LAST ARGUMENT.


# GREEDV1

GREEDV1 represents the initial iteration of our proposed agent designed to navigate the challenges of the GreedMode. This version is structured into two principal components: one dedicated to the shopping phase and the other focused on the battle phase.

## SHOPPING PHASE

During the shopping phase, the agent's logic unfolds in a sequential manner. First, the agent's knowledge base is enhanced with details about the enemies it  encounters in the battlefield. Positioned at the bottom-left corner of the shop, the agent utilizes its knowledge to make informed decisions about which items to purchase.

Upon completition of the shopping, the agent exits the shop through a locked door at the top-right corner. A skeleton key placed in the shop facilitates thie exit. The inclusion of a locked door serves to safeguard against enemies infiltrating the corridor connecting the battlefield with the shop or entering the shop itself. 

After going through the first door, using the key, the agent switches to a simple cyclic plan, where it just goes through the corridor and reaches, unlocks and opens the second door.

The shopping phase concludes after reaching the first spot outside of the second door.
### Shopping planning

The agent plans what items to buy based on the type of enemies, its money and its health. By tweaking the healthiness and richness of the agent the type and quantity of purchased items varies. Inparticular, the agent knows it is healthy when its health is above the **healthiness level** and knows it has a lot of money when its money is above the **richness level** (both are **hyperparameters** given to the agent).


- when it isn't healthy it buys the best healing item it can buy.


- it purchases the best available weapon it can buy in two cases:
    1. there is an enemy that is not beatable with its current weapon.
    
    2. or it is rich and there is a better weapon in the shop.


&nbsp;

To understand if a monster is beatable or not, the agent keeps track of the presence of the monsters on that level and the power of the monsters it can encounter:

- each monster is assigned a certain level of power.

- each weapon has a power level.

Thus, when the agent want to know wheter a monster is beatable with the current weapon, or which available weapon is the strongest, it just compares the power level of the two.


#### Getting to the battlefield

After shopping, in order for the agent to get to the battlefield, it has to: 

* **Pick up** the skeleton key, which allows to open any closed door with a certain probability;

* **Open** the door of the shop: since the skeleton key could fail, the agent tries until it suceeds;

* **Traverse the corridor**: the corridor always has a reversed S shape, thus traversing it just consists in a cyclic plan of going East -> South -> East until it bumps against something;

* **Open** the door to the battlefield, mirroring the action it took with the door in the shop.

## BATTLE PHASE

When the agent reaches the starting point for the battle phase, which is right on the other side of the second door, it starts the battle phase. In this phase, the agent's primary objective is to collect all available gold on the map while dealing with the encountered enemies. It focuses on maximizing its gold acquisition before advancing to the next level.

In the battlefield phase, the agent faces a notable challenge: while it possesses knowledge of the positions of all the gold and the stairs leading to the next level, it remains unaware of the current locations of the enemies. This necessitates a level of adaptability and flexibility in the agent's approach, as it must contend with unforeseen situations not previously calculated. To approach this dynamic scenario, the agent employs a planning and decision-making mechanism. 

### Battle planning and execution

Upon entering the battlefield, the agent faces the critical decision of determining which cluster of gold to approach. The computational complexity of calculating exact distances to all possible gold locations necessitates a more efficient approach. Consequently, the agent opts for a cost-effective strategy by computing the Chebyshev distance between its current position and all potential gold locations. Once these distances are determined, the agent selects the closest gold based on this metric and devises its route towards the chosen gold using the A* algorithm, using the Chebyshev distance.

While there might be arguments against the use of the Chebyshev distance as the optimal heuristic in this particular scenario, such assertions are unfounded. It is crucial to recognize that in the context of NetHack, the agent's movement mirrors that of a king in chess. The Chebyshev distance captures the number of steps a king would take on an empty chessboard to reach a specified position. This alignment between the agent's movement and the Chebyshev distance establishes its efficacy as a heuristic for guiding the A* algorithm. 

In particular, it's noteworthy that alternative common heuristics employed in A*, such as the Euclidean distance or the Manhattan distance, are not admissible in this specific scenario. Admissibility is a critical criterion for heuristics in A*, ensuring that the heuristic never overestimates the cost to reach the goal. In contrast, both the Euclidean and Manhattan distances, when applied as heuristics in this context, lead to inadmissible results.

While planning using A* gives us an optimal path towards the chosen goal, the complexity of the environment demands a more sophisticated decision-making process. Blindly advancing towards the gold without regard for the surroundings is indeed impractical. Thus, at each step, we have the agent evaluate whether to adhere to the original plan or prioritize an alternative action. This strategic consideration stems primarily from the presence of enemies within the dynamic scenario. When the agent finds itself in proximity to an enemy, it deviates from the original plan, engaging in combat to neutralize the encountered threat. Only after successfully dispatching the enemies and returning to a secure state does the agent embark on the process of replanning its path.

### Parameters settings

In [ ]:
#settings for the agent parameters
NUM_EPISODES = 5
MAX_STEPS = 100
PATH = 'kb.pl'
RISK_THRESHOLD=50# when the agent feels it's not healthy anymore
MONEY_THRESHOLD=1
MONEY=20
ARROWS= 2
ARROWS_THRESHOLD=4
#initialize knowledge base
KB = Prolog()
KB.consult(PATH)

### Vacuum experiment

In [ ]:
def vacuum_experiment(des_file,room_type,starting_money,starting_health = 18,starting_items=[],debug = False,show_episode=False) :

    rewards = []
    action=None

    #ran=run.Run()
    for _ in range(1):

        #set agent parameters
        KB.asserta(f"health_threshold({RISK_THRESHOLD})")
        KB.asserta(f"money_threshold({MONEY_THRESHOLD})")
        KB.asserta(f"money({starting_money})")
        KB.retractall('onPlan(_)')
        KB.retractall('battle_begin')
        #des_file=ran.getdes()
        #room_type=ran.gettype()
        
        env = gym.make("MiniHack-Skill-Custom-v0",
                        character="Agent-cav-hum-neu-mal",
                        observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
                        wizard = True,
                        max_episode_steps = MAX_STEPS ,
                        options =(
                            "color",  # Display color for different monsters, objects, etc
                            "showexp",  # Display the experience points on the status line
                            "nobones",  # Disallow saving and loading bones files
                            "nolegacy",  # Not display an introductory message when starting the game
                            "nocmdassist",  # No command assistance
                            "disclose:+i +a +v +g +c +o",  # End of game prompt replies
                            "runmode:teleport",  # Update the map after movement has finished
                            "mention_walls",  # Give feedback when walking against a wall
                            "nosparkle",  # Not display sparkly effect for resisted magical attacks
                            "showscore",  # Shows approximate accumulated score on the bottom line
                            "race:dwa","gender:mal","align:neu","role:cav",
                            "!autopickup",
                            "pettype:none",
                        ),
                        des_file=des_file,
                )
        obs=env.reset(wizkit_items = starting_items + ['20 scroll of fire','6 potion of confusion','unicorn horn'])
        if debug: display_inventory(obs['inv_strs'])
# elabora in base alle osservazioni presenti
        


        #env.render()
        #print(des_file)
        # count the number of steps of the current episode
        steps = 0
        # store the cumulative reward
        reward = 0.0
        # collect obs['pixel'] to visualize
        ep_states = []
        #agent starts in the shop
        in_battle = False
        #agent start with no plan for battle
        planned_actions = False
    

        #process level information
        
        if any(map(lambda x:x==room_type,['mazeroom','minibossroom'])):
            KB.asserta("corridors")
        monsters=extract_monsters(des_file)

        #compute the second closed door position
        goal_x,goal_y=-1,-1
        level_heigth=len(obs['screen_descriptions'])
        level_width=len(obs['screen_descriptions'][0])
        for i in range(level_heigth):
            for j in range(level_width):
                objs = bytes(obs['screen_descriptions'][i][j]).decode('utf-8').rstrip('\x00')
                if 'closed door' in objs and i > goal_x:
                    goal_x,goal_y=i,j
        KB.asserta(f'battlefield_start({goal_x},{goal_y})')

        #print(f'battlefield_start({goal_x},{goal_y})')
            
        ep_states.append(obs['pixel'])
        done = False

        #set health to starting health (max 16)
        obs, reward, done, info= set_health(env,obs,min(starting_health,18))
        #display_inventory(obs['inv_strs'])

        

        #---- obtain information about gold and stairs down of the level ------#
        game_map = obs['chars']
        game = obs['pixel']

        #retrieve map informations
        amulet_des=get_reference_des(des_file)
        amulet_map = get_target_location(game_map,symbol='"')
                
        stair_des_pos = get_stair_location(des_file)

        #when looking at des to get information, the position of the items in the real envirment is translated by a certain vector
        delta_x, delta_y = (amulet_map[0]-amulet_des[0],amulet_map[1]-amulet_des[1])


        gold_pos_list = get_money_location(des_file)

        gold_pos_list = [(x+delta_x,y+delta_y)for (_,(x,y)) in gold_pos_list] #modify here if interested in gold amount on ground

        #tell to KB gold positions
        for (x,y) in gold_pos_list :
            KB.asserta(f'position(gold,_,{x},{y})')
        
        #tell to KB stairs down location
        target = (stair_des_pos[0]+delta_x,stair_des_pos[1]+delta_y)
        KB.asserta(f'position(stairs,_,{target[0]},{target[1]})')

        #------ end level processing ------------#

        #---- initialize metric to measure ------#
        metric=dict()
        metric["steps"]=0
        metric["damage"]=0
        metric["money_gain"]=0
        metric["money_loss"]=0
        metric["healing_count"]=0
        # Main loop
        while not done and steps < MAX_STEPS:
            # proccess current state
            process_state(obs, KB)

            #check if process state has seen that we are in the position to start battle
            if (not in_battle) and len(list(KB.query('battle_begin'))) > 0 : #if yes switch to battle mode
            
                in_battle = True
                #retrieve money amount
                

            #only done to identify monster position on 
            if not in_battle :
                for monster_name,pos_x,pos_y in monsters:
                    KB.asserta(f"position(enemy,\'{monster_name}\',{pos_x},{pos_y})")
            #ask knowledge base the action
            try:
                #print(list(KB.query("position(Obj,Name,R,C)")))
                actions=list(KB.query('action(X)'))
                action = actions[0]
                action = action['X']
            except Exception as e:
                env.render()
                raise e                
                #print("no action available")
                #print(list(KB.query("position(object,Type,R,C)")))
                print(e)
                action = None
            # Perform the action in the environment
            #print(f'ACTION:\t {action}')

            #perform the action and compute metrics
            if action:
                # gathering info for metric measurement
                agent_r=obs['blstats'][1]
                agent_c=obs['blstats'][0]
                agent_health=int(obs['blstats'][10])
                agent_money= int(list(KB.query("money(M)"))[0]["M"])

                obs, reward, done, info ,planned_actions = perform_action(action, env,KB,planned_actions,obs)
                if debug: env.render()
                #compare old state with the new one for metric measurments
                agent_new_money=int(list(KB.query("money(M)"))[0]["M"])
                agent_new_health= int(obs['blstats'][10])
                if not done :
                    if agent_r - obs['blstats'][1] != 0 or agent_c - obs['blstats'][0] != 0:
                        metric["steps"]+=1
                    if agent_health > agent_new_health:
                        metric["damage"]+=agent_health - agent_new_health
                    elif 'quaff' in action or 'eat' in action :
                        metric["healing_count"]+=1
                    if agent_new_money > agent_money:
                        metric["money_gain"]+= agent_new_money - agent_money
                    elif agent_new_money < agent_money:
                        metric["money_loss"]+= agent_money - agent_new_money
                
                ep_states.append(obs['pixel'])
                #if debug: env.render()
                #env.render()
            else:
                break
            steps += 1

            is_dead, final_obs, final_reward, final_done, final_info = check_death(env,obs)
            if is_dead :
                obs = final_obs
                reward = final_reward
                done = final_done
                info = final_info

        
        if steps == MAX_STEPS:
            print("reached maximum amount of steps")
        # Display game with interface
        #show_match(ep_states)
        rewards.append(reward)
        
        # reset the environment and retract axioms that may cause errors
        #print(list(KB.query('battle_begin')))
        #print(list(KB.query('position(stairs,_,X,Y)')))
        metric["health"]= int(obs['blstats'][10])
        metric["total_money"]= int(list(KB.query("money(M)"))[0]["M"])
        
        last_inventory = list(filter(lambda x:x!='',map(lambda x: bytes(x).decode('utf-8').rstrip('\x00'),obs['inv_strs'])))

        #obs = env.reset(wizkit_items = ['healing potion','20 scroll of fire','6 potion of confusion','unicorn horn'])
        KB.retractall('stepping_on(agent,_,_)')
        KB.retractall('position(_,_,_,_)')
        KB.retractall('shopping_done')
        KB.retractall('corridors')
        KB.retractall('battle_begin')
        KB.retractall('battlefield_start(_,_)')
        KB.retractall('money(_)')
        #ran.nextlevel()

    #print(f'After {NUM_EPISODES} episodes, mean return is {sum(rewards)/NUM_EPISODES}')
    #print("The rewards of the episodes are:", rewards)
    if debug: display_inventory(obs['inv_strs'])
    if show_episode: show_match(ep_states)
    return metric, info, last_inventory

### State processing 



The state processing gathers informations from the environment and puts them into the knowledge base, if it is in battle phase, signaled by the `in_battle` flag, it also retains information about the gold ans the stairs in the battlefield.

The processing is done by the `process_state` function, that takes as arguments the observations of the environment, the knowledge base and the in_battle flag, and updates the `KB` accordingly.

First the function goes trough the level tiles and using the screen description inserts the following facts in the knowledge base:

- the position of any object available in the shop with it's name and the cost

- the position of any enemy visible
- the position of the locked door and the key to unlock it
    - if the door is unlocked it is inserted as a tile so the agent can go through it

When it has analized the environment the function observes the agent inventory and refreshes the knwoledge base on what it has, only some objects are of our interest: if they are not specified in run module they are not considered.

Following the inventory check, the function proceeds to examine whether the agent has reached the battlefield. If this condition is met, a signal `battle_begin` is relayed to the knowledge base, indicating the start of the battle phase. 

Subsequently, the function processes the received message, which serves as a valuable source of information regarding the agent's interactions and the consequences of its actions.

In [ ]:
def process_state(obs: dict, kb: Prolog, in_battle:bool = True):

    #to mantain global information about gold positions
    if in_battle :
       # print('QUAAAAAa')
        gold_pos_list= list(kb.query("position(gold,_,X,Y)"))
        #print(gold_pos_list)
        stairs_pos = list(kb.query("position(stairs,_,X,Y)"))[0]
        #print(stairs_pos)

    kb.retractall("position(_,_,_,_)")

    if in_battle :
        for gold_pos in gold_pos_list : #put the information about gold that you have eliminated
            gold_x = int(gold_pos['X'])
            gold_y = int(gold_pos['Y'])
            kb.asserta(f'position(gold,_,{gold_x},{gold_y})')

        kb.asserta(f'position(stairs,_,{int(stairs_pos["X"])},{int(stairs_pos["Y"])})') #putting information about stairs again, even if not in observation


    # elabora in base alle osservazioni presenti
    level_heigth=len(obs['screen_descriptions'])
    level_width=len(obs['screen_descriptions'][0])

    for i in range(level_heigth):
        for j in range(level_width):
            if (obs['screen_descriptions'][i][j] == 0).all(): continue
            objs = bytes(obs['screen_descriptions'][i][j]).decode('utf-8').rstrip('\x00') 
            obj_type=chr(obs['chars'][i][j]) # characters indicates the type of object
            if obj_type == '!':
                kb.asserta(f"position(potion,healing,{i},{j})")
                kb.asserta(f"available_to_buy(potion,healing,5)")
            elif obj_type == ')':
                arrow= list(filter(lambda x:x in objs,AVAIABLE_ARROWS))
                if len(arrow)!=0:# if the list isn't empty it's an arrow otherwise it's a weapon
                    arrow=arrow[0]
                    kb.asserta(f"position(arrow,\'{arrow}\',{i},{j})")
                    kb.asserta(f"available_to_buy(arrow,\'{arrow}\',2)")
                weapon= list(filter(lambda x:x in objs,AVAIABLE_WEAPONS))
                if len(weapon)!=0 :
                    weapon = weapon[0]
                    kb.asserta(f"position(weapon,\'{weapon}\',{i},{j})")
                    kb.asserta(f"available_to_buy(weapon,\'{weapon}\',15)")
                #special case: the katana appears as a samurai sword until it's stepped on
                if 'samurai sword' in objs:
                    kb.asserta(f"available_to_buy(weapon,\'katana\',15)")
                    kb.asserta(f"position(weapon,\'katana\',{i},{j})")
            elif obj_type == '[': 
                armor= list(filter(lambda x:x in objs,AVAIABLE_ARMORS))
                if len(armor)!=0:
                    armor=armor[0]
                    kb.asserta(f"position(armor,\'{armor}\',{i},{j})")
                    kb.asserta(f"available_to_buy(armor,\'{armor}\',20)")
            elif 'closed door' in objs:
                kb.asserta(f"position(object,door,{i},{j})")
            elif 'open door' in objs or 'floor' in objs:
                kb.asserta(f"position(object,tile,{i},{j})")
            elif objs in MOB or objs in MINIBOSS or objs in BOSS:
                #print(obj)
                kb.asserta(f'position(enemy,\'{objs}\',{i},{j})')
            elif 'key' in objs :
                kb.asserta(f"position(tool,'skeleton key',{i},{j})")
            elif 'runed broadsword' in objs:
                kb.asserta(f"position(weapon,'elven broadsword',{i},{j})")
            elif 'stout spear' in objs:
                kb.asserta(f"position(weapon,'dwarvish spear',{i},{j})")
            

    

    
    kb.retractall("wields_weapon(_,_)")
    kb.retractall("has(agent,_,_)")

    #display_inventory(obs['inv_strs'])

    for item in obs['inv_strs']:
        item = bytes(item).decode('utf-8').rstrip('\x00')
        if 'weapon in hand' in item:
            # the actual name of the weapon is in position 2
            # qualcosa nomearma (weapon in hand)
            wp = " ".join(item.split()[1:]).split('(')[0].rstrip()
            #print(f'Arma: {wp}')
           
            kb.asserta(f"""wields_weapon(agent,'{wp}')""")

            objs=list(filter(lambda x: x in item,AVAIABLE_WEAPONS))
            if len(objs)!=0:
                wp=objs[0]

            if wp not in AVAIABLE_WEAPONS :
                kb.retractall(f"""weapon_stats('{wp}',_)""")
                kb.assertz(f"""weapon_stats('{wp}',0)""")
            continue


        inv_item=list(filter(lambda x:x in item,AVAIABLE_WEAPONS))
        if len(inv_item)!=0 :
            kb.asserta(f"has(agent,weapon,\'{inv_item[0]}\')")

        inv_item=list(filter(lambda x:x in item,AVAIABLE_ARMORS))
        if len(inv_item)!=0 :
            kb.asserta(f"has(agent,armor,\'{inv_item[0]}\')")

        inv_item=list(filter(lambda x:x in item,AVAIABLE_ARROWS))
        if len(inv_item)!=0 :
            kb.asserta(f"has(agent,arrow,\'{inv_item[0]}\')")

        inv_item=list(filter(lambda x:x in item,AVAIABLE_OBJECTS))
        if len(inv_item)!=0 :
            if 'potion' in inv_item[0]:
                kb.asserta(f"has(agent,potion,\'healing\')")
        if 'potion' in item:
            kb.asserta(f"has(agent,potion,\'healing\')")
        

    # processa in base allo stato dell'agente

    # aggiornamento posizione agente e vita
    kb.retractall("health(_)")
    kb.asserta(f"position(agent, _, {obs['blstats'][1]}, {obs['blstats'][0]})")
    kb.asserta(f"health({int(obs['blstats'][10]/obs['blstats'][11]*100)})")
    agent_r=obs['blstats'][1]
    agent_c=obs['blstats'][0]

    bs=list(kb.query("battlefield_start(R,C)"))[0]

    if agent_r == bs['R'] and agent_c == bs['C']+1: #even if it goes on this specific cell multiple times, it doesn't change anythign
        kb.asserta("battle_begin")

    kb.retractall('stepping_on(agent,_,_)')

    # processa messaggio sullo schermo
    message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
    #print(f'MESSAGE:{message}')
    if 'The door opens' in message:
        kb.asserta("shopping_done")
    if 'You see here' in message:
        armor=list(filter(lambda x:x in message,AVAIABLE_ARMORS))
        if len(armor) !=0:
            kb.asserta(f'stepping_on(agent, armor, \'{armor[0]}\')')
        weapon=list(filter(lambda x:x in message,AVAIABLE_WEAPONS))
        if len(weapon) !=0:
            kb.asserta(f'stepping_on(agent, weapon, \'{weapon[0]}\')')
        arrow=list(filter(lambda x:x in message,AVAIABLE_ARROWS))
        if len(arrow) !=0:
            kb.asserta(f'stepping_on(agent, arrow, \'{arrow[0]}\')')
        objs=list(filter(lambda x:x in message,AVAIABLE_OBJECTS))
        if len(objs) !=0:
            kb.asserta(f'stepping_on(agent,potion,healing)')
        if 'potion' in message:
            kb.asserta(f'stepping_on(agent,potion,healing)')
        if 'key' in message:
            kb.asserta("""stepping_on(agent,tool,'skeleton key')""")
        #""" if 'stout spear' in message:
        #    kb.asserta(f'stepping_on(agent, weapon, \'stout spear\')') """

        #model stepping_on gold
    if in_battle :
        for gold_pos in gold_pos_list : #put the information about gold that you have eliminated
            gold_x = int(gold_pos['X'])
            gold_y = int(gold_pos['Y'])
            if gold_x == obs['blstats'][1] and gold_y == obs['blstats'][0] :
                kb.asserta('stepping_on(agent,gold,_)')

### Performing an action




After the state processing the agent will ask the knowledge base what is the next action to do and perform it. The agent first queries the knowledge base for the action(X) predicate, where X is the action to execute, that is passed as an argument to `perform_action`. There are different possible action, and they also depend on wheter the agent is in the shopping or battle phase.

Some actions are very simple, and consists only of the execution of an atomic step by choosing the correct `action_id` and invoking the `env.step(action_id)` function to make that move. An example of this are the final movement actions or the `pick key`action.


Other functions are complex and require additional work to be performed:

- `eat`, `quaff` and `wield` work in the same way: first perform the eat/quaff action then select the last object available for that action. This is because the actions are alway performed as first whenever possible, guaranteeing the interested item is in the last slot for their respective promp.

- `attack(Direction)` will attack in the direction specified by Direction and discard the previous plan of the agent.
- `followPlan(Direction)` takes a step in the direction specified by the plan.
- `plan` selects the closest gold (if available) or the stairs as the goal and executes A* to plan the path to reach it, taking also the first step of the path.
- `apply` is used to apply the key to open the locked door.
- `pick` is used to pickup gold , when the action is done the money in the knowledge base are updated.

- `buy item(Type,Name,Cost)` is used to buy an item. In the game the agents perform just a pick but the knowledge base gets updated with the new money the agent has after the purchase.

In [ ]:
def perform_action(action, env,kb,planned_actions,obs):

    name,args=parse_predicate(action)
    if name == 'eat': 
        action_id = 29
        # print(f'Action performed: {repr(env.actions[action_id])}')
        obs, _, _, _ = env.step(action_id)
        # Example message:
        # What do you want to eat?[g or *]
        message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        food_char = message.split('[')[1][0] # Because of the way the message in NetHack works
        action_id = env.actions.index(ord(food_char))

    elif name == 'quaff' :
        action_id = 52
        obs, _, _, _ = env.step(action_id)

        message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        food_char = message.split('[')[1][0] # Because of the way the message in NetHack works
        action_id = env.actions.index(ord(food_char))
        
    elif name == 'attack' :
        #print(action)
        action_id = DIR_TO_NUM_MAP[args[0]]
        kb.retractall('onPlan(_)')

    elif 'followPlan' == name :
        action_id = DIR_TO_NUM_MAP[args[0]]
        kb.retractall('plannedMove(_)') #remove last planned move
        if len(planned_actions) > 0 :
            kb.asserta(f'plannedMove({NUM_TO_DIR_MAP[planned_actions.pop(0)]})')
        else : kb.retractall('onPlan(_)')

    elif 'plan' in action : #have to formulate plan
        kb.retractall('plannedMove(_)') #remove past planned move

        #get agent pos
        agent_pos = list(kb.query('position(agent,_,X,Y)'))[0]
        start = (agent_pos['X'],agent_pos['Y'])

        #get stairs pos
        stair_pos = list(kb.query('position(stairs,_,X,Y)'))[0]
        target = (stair_pos['X'],stair_pos['Y'])

        #get all gold pos
        gold_pos_list= list(kb.query('position(gold,_,X,Y)'))

        gold_pos_list = [(element['X'],element['Y'])for element in gold_pos_list]

        gold_pos = get_closest_gold(gold_pos_list,start)
        #print(f'Gold at:{gold_pos}')

        if gold_pos is None :
            gold_pos = target

        game_map = obs['chars']

        path,_ = a_star(game_map, start, gold_pos,chebyshev_distance)
        #print(f"PATH:{path}")
        planned_actions = actions_from_path(start, path[1:])

        action_id = planned_actions.pop(0) #retrieve action to execute


        #insert next planned move in kb
        if len(planned_actions) > 0 : #if plan made of one action (thus len == 0 here) we need to replan next
            kb.asserta(f'plannedMove({NUM_TO_DIR_MAP[planned_actions.pop(0)]})')
            kb.asserta('onPlan(agent)') #tell that new plan is available

    elif 'pick_key' == action :
        action_id = 49
    elif name == 'apply' :
        while True:
            action_id = 20
            obs, _, _, _ = env.step(action_id)
            #env.render()

            #roba del tipo: What do you want to use or apply? [fg or ?*]
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
            weapon_char = message.split('[')[1].split(' ')[0][-1]
            action_id = env.actions.index(ord(weapon_char))

            obs, _, _, _ = env.step(action_id)
            #env.render()
            #appare messaggio: In what direction?
            obs, _, _, _ = env.step(DIR_TO_NUM_MAP[args[0]])
            #env.render()

            #appare messaggio
            # Unlock it? [yn] (n)
            obs, _, _, _ = env.step(env.actions.index(ord('y')))
            #env.render()

            # You succeed in unlocking the door.
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
            if 'You succeed in unlocking the door.' in message :
                action_id = int(DIR_TO_NUM_MAP[args[0]]) #open the door
                break 

    elif 'pick' in action: #picking gold
        action_id = 49
        kb.retractall('stepping_on(agent,_,_)')
        agent_pos = list(kb.query('position(agent,_,X,Y)'))[0]
        agent_x = int(agent_pos['X'])
        agent_y = int(agent_pos['Y'])
        kb.retractall(f'position(gold,_,{agent_x},{agent_y})')

        #updating money
        #retrieve money before picking up gold
        game_money_old = int(obs['blstats'][13])
        curr_money = int(list(kb.query('money(X)'))[0]['X'])
        
        #perform action
        #print(f'>> Current action from Prolog: {action}')
        obs, reward, done, info = env.step(action_id)

        #obtain new game money
        game_money_new = int(obs['blstats'][13])
        #update kb
        kb.retractall('money(_)')
        kb.asserta(f'money({curr_money + game_money_new - game_money_old})')

        return obs, reward, done, info , planned_actions
    
    elif name == 'wield':
        action_id = 78
        obs, _, _, _ = env.step(action_id)
        #env.render()

        # Example message:
        # What do you want to wield? [- abcdg or ?*]

        message = bytes(obs['message']).decode('utf-8').rstrip('\x00')

        weapon_char = message.split('[')[1].split(' ')[1][-1]

        action_id = env.actions.index(ord(weapon_char))

    elif name== 'buy_item':
        action_id = 49
        cost=int(args[0])
        item_type=args[1]
        item_name=args[2]
        kb.retractall('stepping_on(_,_,_)')
        kb.retractall('available_to_buy(_,_,_)')
        #refersh money after purchase
        old_money=int(list(kb.query('money(X)'))[0]['X'])
        kb.retractall("money(_)")
        kb.asserta(f"money({old_money-cost})")
        # if it's buying arrows it will update the number of arrows the agent has
        if ('armor' in item_type):
            healthiness=int(list(kb.query('health_threshold(X)'))[0]['X'])
            protection= int(list(kb.query(f"armor_stats(\'{item_name}\',X)"))[0]['X'])
            kb.retractall("health_threshold(X)")
            kb.asserta(f"health_threshold({protection*0.25 + healthiness})")
        if('arrow' in item_type):
            old_arrows=int(list(kb.query('arrows(X)'))[0]['X'])
            kb.retractall("arrows(X)")
            kb.asserta(f"arrows({old_arrows+1})")
        kb.asserta(f"has(agent,\'{item_type}\',\'{item_name}\')")

    # Movement/Attack/Run/Get_To_Weapon actions
    # in the end, they all are movement in a direction
    elif 'northeast' in action: action_id = 4
    elif 'southeast' in action: action_id = 5
    elif 'southwest' in action: action_id = 6
    elif 'northwest' in action: action_id = 7
    elif 'north' in action: action_id = 0
    elif 'east' in action: action_id = 1
    elif 'south' in action: action_id = 2
    elif 'west' in action: action_id = 3
    elif 'sit' in action : action_id = 11
    obs, reward, done, info = env.step(action_id)

    if name == 'apply' :
        message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        #the door resist
        while 'The door opens.' not in message and not done:
            #env.render()
            obs, reward, done, info = env.step(action_id)
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
    return obs, reward, done, info , planned_actions


### DEMO
by executing the the cell below you will be able to see the simulations for a single level.

In [ ]:
map_des,map_type=completemapdes(2,1)
vacuum_experiment(map_des,map_type,20,18,[],False,True)
pass

## Performing a game
To perform multiple levels there some problems to take into account:

1. it is difficult to create a single episode with multiple levels , so we had to create a new environment each time. In doing so the items from the previous level get erased, so we had to use the special **wizard mode**, that allows to give the player any item we want. Thus, each time a level is cleared, we take from the old level the last inventory of the player and give the items to player in the new level.

2. nle and minihack don't give the possibility to set the agent health therefore we had to find another way to set the agent health using the `set_health` function.


To perform a series a vacuum experiments we use the `simulate_runs` function, based on the following loop:
- for each episode :
    - generate a new level
    
    - run the level with the vacuum experiment function
    - get the information about the level performance
    - store the items you want to pass into the next level along with other parameters such as health and money
    - store the metrics for this level
    - if the player dies or terminates all five levels store the metrics accumulated and go on to the next episode
- when al the episodes are done return the the list containing the metrics for each episode

In [ ]:
def simulate_runs(starting_point,n):
    simulations=[]
    i = starting_point
    for seed in range(starting_point,n):
        ran = run.Run(seed=seed)
        all_metrics = []
        all_info = []
        money = MONEY
        health = 18 
        items=['healing potion']
        debug = False
        try:
            for l in range(5):
                print(f'\tLEVEL:{l+1}\tSIMULATION:{i}')
                #if i == 2: debug = True
                metric, info, last_inv = vacuum_experiment(ran.getdes(),ran.gettype(),money,starting_health=health,starting_items=items, debug=False)
                new_inv=[]
                for item in last_inv:
                    objs=list(filter(lambda x: x in item,run.AVAIABLE_ARMORS))
                    if len(objs)!=0:
                        new_inv.append("an "+item[0])
                    list(filter(lambda x: x in item,run.AVAIABLE_OBJECTS))
                    if len(objs)!=0:
                        new_inv.append(item[0])
                    list(filter(lambda x: x in item,run.AVAIABLE_WEAPONS))
                    if len(objs)!=0:
                        new_inv.append("an "+item[0])
                    if 'potion' in item:
                        pn=item.split(' ')[0]
                        n = 1 if 'a' in pn else int(pn)
                        new_inv.append(str(n)+" potion of healing")

                new_inv.append(list(KB.query("wields_weapon(agent,Name)"))[0]['Name'])
                items=new_inv
                all_metrics.append(metric)
                all_info.append(info)
                health = metric['health']
                money = metric['total_money']
                if health == 0 : break
                ran.nextlevel()
            
            simulations.append(all_metrics) 
        except Exception as e:
            print(f"SIMULATION no.{i+1} crashed")
        i +=1
    return simulations

Now we explain the solution to our second problem: setting the health of the agent.

The problem is solved by the `set_health` function which sets the agent health to its appropriate value.

To do so the function uses the following trick: in nethack, when the agent is confused, if it's reading the scrool of fire it take **1 damage**, so we make the agent drink the potion of confusion using the `make_confusion` function and read the scroll of fire the right number of times, that is the difference between target health and actual health, we can set the agent health to our liking.

When the agent is done with the scrolls it waits for the effect of the potion to go away and then it drops the scrolls and potion of confusion, starting the simulation as planned.

In [ ]:
def set_health(env,obs, target_health, max_health = 18) :
    #until not confuso -> drink potion of confusion
    #read scroll until health is right
    #use unicorn horn
    #drop potions
    #drop scroll
    if target_health != max_health  : #only have to do if we want to reduce healt

        obs = make_confuse(env)
        if max_health < target_health or int(obs['blstats'][10])  < target_health:
            raise Exception("max_health or current health less than target")
        while int(obs['blstats'][10]) > target_health : #current health grater than target
            
            obs,_,_,_ = env.step(54)
            #env.render()

            #What do you want to read? [f or ?*]
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00') #parse just life drinking

            if 'You feel less confused now.' in message : #means he has to be confused again
                env.step(env.actions.index(ord('a'))) #in pos 'a' there's the weapon of the agent, so he can't read it
                #env.render()
                obs = make_confuse(env)
                continue #need to start from reading again
            
            
            scroll_char = message.split('[')[1].split()[0][-1] #guaranteed to be the last since we are the ones deciding the ordering

            obs,_,_,_ = env.step(env.actions.index(ord(scroll_char)))
            #env.render()

            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')

            if 'You feel less confused now.' in message : #means he has to be confused again
                obs = make_confuse(env)
                continue #need to start from reasing again
        
        # PROVA 
        if message == None:
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        #we have reached target health
        #use unicorn horn to remove confusion
        while 'You feel less confused now.' not in message :
            obs, _, _,_ = env.step(20) #apply
            #env.render()

            #there's a similar prompt to the ones before
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
            unicorn_char = message.split('[')[1].split()[0][-1]
            #print(message)

            obs,_,_,_ = env.step(env.actions.index(ord(unicorn_char))) #applies unicorn horn
            #env.render()
            message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        

    #now we have to drop items
    #problem: either confusion passed by itself or by applying unicorn horn (it' a roll everytime for it to work)
    items_to_remove = 0
    for item in obs['inv_strs']:
        item = bytes(item).decode('utf-8').rstrip('\x00')
        if 'unicorn horn' in item : #drop last item
            items_to_remove += 1
        elif 'of confusion' in item:
            items_to_remove += 1 
        elif 'scroll' in item :
            items_to_remove += 1

    if target_health == max_health :
        items_to_remove = 3

    for _ in range(items_to_remove) :
        obs,_,_,_ = env.step(27)
        #env.render()

        #message similar to other ones
        message = bytes(obs['message']).decode('utf-8').rstrip('\x00')
        last_item_char = message.split('[')[1].split()[0][-1]

        obs, reward, done, info = env.step(env.actions.index(ord(last_item_char)))
        #env.render()

    return obs , reward, done ,info

# GREEDV2

The second agent has the following enchancements:


- it can now buy and equip armors


- the shopping decision-making is improved
- when endangered in battle the agent now prefers to go to the end of the level


## The second agent shopping phase

In contrast to GREEDV1, GREEDV2 now considers the purchase of armors during the shopping phase. To effectively utilize armors, the agent follows a specific sequence: it begins by unwearing the current armor, proceeds to equip the desired armor, and ultimately discards the unused armor. This approach is motivated by the weight associated with armors, which can potentially burden the agent(https://nethackwiki.com/wiki/Encumbrance).

In the enhanced evaluation of equipment, which encompasses both armors and weapons, GREEV2, when facing unbeatable enemy, defined as having a power level exceeding the sum of the power of the wielded weapon and the worn armor. Now the agent selects the equipment that offers a more substantial increase in its overall power. This decision-making process is made possible by assigning a specific power value to both armors and weapons in the knowledge base.


Furthermore, In GREEDV2, when surpassing the richness threshold, the agent prioritizes acquiring healing potions. This strategic choice enhances sustainability, particularly valuable in tackling the further levels.


## The second agent battle phase


In the battle phase, GREEDV2 introduces a specific adjustment to the agent's behavior. Unlike GREEDV1, when the agent is not in optimal health and needs to strategize, it now prioritizes moving towards the stairs (the exit) rather than heading directly to the closest gold. This shift aims to minimize engagements with enemies, allowing the agent to evade potential threats and engage less in combat for improved survivability.

#### **NOTE** : due to pyswipl not supporting multiple kb,  to execute the GREEDV2 example, it is needed to RESTART THE KERNEL

In [ ]:
#BEFORE EXECUTION:

#Due to pyswipl impossibility in supporting multiple kb, to execute the GREEDV2 example, it is needed to RESTART THE KERNEL

import gym
import minihack
from run import *
from algorithms import  a_star
from greedv2 import vacuum_experiment

from utils2 import process_state, perform_action,extract_monsters, set_health, check_death, display_inventory,parse_predicate,show_match
from utilsbattle2 import actions_from_path,get_money_location, get_stair_location, get_target_location, get_reference_des,NUM_TO_DIR_MAP,DIR_TO_NUM_MAP,get_closest_gold,chebyshev_distance

from pyswip import Prolog
#initialize knowledge base
KB = Prolog()
KB.consult("kb2.pl")

In [ ]:
map_des,map_type=completemapdes(1,1)
vacuum_experiment(map_des,map_type,20,KB,18,[],False,True)
pass

# Performance evaluation

To evaluate the performances of the agent, as stated in the introduction, we mainly use three metrics: the **healthiness**, the **richness** and the **levels completed**: 


* **Healtiness**: the remaining number of hit points. To restore them the shop provides health potions and armors to preserve them.

  
* **Richness**: the remaining amount of money at the agent's disposal. A proficient agent can strategically manage its resources, avoiding the necessity to purchase the best items at every opportunity.

  
* **Levels completed**: the last metric that is important to us are the levels completed by the agent. Indeed, we have to keep in mind, while we want to preserve our health and have a large quantity of money, those are mainly than a means to reach higher levels.


In order to effectuate the measurements we had to fix various parameters:

* the agent always starts a run (of 5 levels) with 20 gold;

* The threshold to decide whether the agent is healthy or not is always 50% of the health (meaning that when the agent's current HP reach half of its maximum HP the agent is not healthy anymmore);

* the measurements for the experiments have been made on 30 runs.

## Comparison between differenth richness threshold

Our first comparison involves in showing the behaviour of the different versions of the agent when changing the richness threshold. As a remainder, when increasing this threshold the agent needs a greater amount of money order to consider itself rich. This could impact the performances of the agent in different ways, since usually, in both of the two versions, when the agent is reach it tends to buy more items (whether equipment or healing potions).

Specifically, the following graphs show the behaviour of the agent with richness threshold **R** = 1 and **R** = 20. We chose R = 1, because, intuitively, since the agent tends to consider himself rich when it has less money, having lower R makes the agent buy more items, thus increasing its sustainability capabilities through the levels.

This consideration is confirmed in the following graph, where GREEDV1 is discussed. Indeed, we can see how the agent with R = 20, in the trials examined, while reaching the same levels as the one with R = 1, it reaches them only in a fewer number of examples.

Moreover, we can see how GREEDV1, in general, fails to complete the last level, dying at the hand of the final boss.

![V1R1R20](Images/V1R1R20.png)

The two here have agents have similar performances and both manage to complete all the levels in some experiments. However here, while the performances in general outclass the ones from GREEDV1, we can see how our initial considerations are not true. Indeed here, the agent with R = 20 is the better one in terms of number of completed levels and total remaining money. One possible justification of such phenomena is that the agent with R = 20 accumulates more money in the early levels, and thus, when reaching the later levels, it spends his money on better equipment, giving him better chances of survival


![V2R1R20](Images/V2R1R20.png)

## Comparison between V1 and V2

Here we showcase, when fixing the richness threshold R, how the two versions of the agent compare to each other.

In the following plot, we can see how, while having a better survivability, GREEDV2 tends to spend a lot more money than GREEDV1. Indeed, considering the introduction of armors, and the changes in item priorities when rich, GREEDV2 buys more items. This is reflected in the number of levels beaten. In fact, GREEDV2 has 4 experiments where it manages to beat level 5, while GREEDV1 always dies. This is also shown by the remaining health of the two agents: GREEDV2 usually has more health than GREEDV1

![V1V2R1](Images/V1V2R1.png)

The general trend seen in the previous plot, comparing GREEDV1 and GREEDV2, is repeated in this last plot, where we compare the two agent versions when fixing R = 20.

![V1V2R20](Images/V1V2R20.png)